In [1]:
# Import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
from dateutil.parser import parse

from datetime import timedelta, date
import numpy as np

In [2]:
countries = [
    'portugal','spain','italy','france','germany','uk',
    'us','iran','china','south-korea',
]

superes = {}

for country in countries:
    
    print (country)

    # Set the URL you want to webscrape from
    url = 'https://www.worldometers.info/coronavirus/country/%s'%country

    # Connect to the URL
    response = requests.get(url)

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text)

    res = pd.DataFrame()

    cases, deaths = [], []
    for ul_tag in soup.find_all('ul', {'class':'news_ul'})[:10]:
        i=0
        for li_tag in ul_tag.find_all('li', {'class':'news_li'}):
            i+=1
            if i==3: break
            strong_tag = li_tag.find_all('strong')
            cases += [strong_tag[0].text.split(' new case')[0]]
            deaths += [strong_tag[1].text.split(' new death')[0]]
    try:
        res['cases'] = [np.nan if (c.lower()==country or c in ['china','South Korea']) else int(c.replace(',','')) for c in cases]
    except:
        res['cases'] = [np.nan for c in cases]
    try:
        res['deaths'] = [np.nan if (d.lower()==country or d in ['china','South Korea']) else int(d.replace(',','')) for d in deaths]    
    except:
        res['deaths'] = [np.nan for c in deaths]

#     dates = []
#     for div_tag in soup.find_all('div'):
#         for d in div_tag:
#             if d.name == 'div' and d.get('class', '') == ['news_date']:
#                 dates += [d.text.split(' (')[0]]
    today = date.today()
    
    res['date'] = [str(d)[:10] 
                   for d in pd.to_datetime([pd.to_datetime(today)-timedelta(x+1)
                                            for x in range(len(cases))], 
                                            format='%Y%m%d')]
    print (res.set_index('date'))
    print ()
    
    superes[country] = res.set_index('date')

portugal
            cases  deaths
date                     
2020-05-28    304      13
2020-05-27    285      14
2020-05-26    219      12
2020-05-25    165      14
2020-05-24    152      14
2020-05-23    271      13
2020-05-22    288      12

spain
            cases  deaths
date                     
2020-05-28   1137       1
2020-05-27    510       1
2020-05-26    859     280
2020-05-25    482      74
2020-05-24    466      50
2020-05-23   1787     688

italy
            cases  deaths
date                     
2020-05-28    593      70
2020-05-27    584     117
2020-05-26    397      78
2020-05-25    300      92
2020-05-24    531      50
2020-05-23    669     119
2020-05-22    652     130

france
            cases  deaths
date                     
2020-05-28    NaN    66.0
2020-05-27    NaN    66.0
2020-05-26    NaN     NaN
2020-05-25    NaN    65.0
2020-05-24    NaN    35.0
2020-05-23    NaN    43.0
2020-05-22    NaN    74.0

germany
            cases  deaths
date                    

In [3]:
xls = pd.ExcelFile("coronav.xls")
cases = xls.parse('cases')[['date']+countries]
deaths = xls.parse('deaths')[['date']+countries]
recovery = xls.parse('recovery')[['date']+countries]
cases['date'] = cases['date'].astype(str)
deaths['date'] = deaths['date'].astype(str)
recovery['date'] = recovery['date'].astype(str)

today = date.today()
lastdate = cases.date.iloc[-1]
iterations = (pd.to_datetime(today)-pd.to_datetime(lastdate)).days

writer = pd.ExcelWriter("coronav.xls", engine='xlsxwriter')
    
for tmp_sheet, name in zip([cases,deaths,recovery],
                           ['cases','deaths','recovery']):
    
    if name!='recovery':
    
        for i in range(1,iterations):
            print (i,name)
            tmp_date = pd.to_datetime(lastdate) + timedelta(days=i)
            tmp_country = [str(tmp_date)[:10]]
            print (tmp_country[0], end=' ')

            for country in countries:
                print (country, end=' ')

                if country not in ['china','south-korea']: 

                    try:
                        tmp_country += [superes[country].loc[tmp_country[0],name]]
                    except KeyError:
                        tmp_country += [np.nan]

                else:
                    tmp_country += [np.nan]

            print ('\n',tmp_country)

            tmp_sheet = tmp_sheet.append(
                pd.DataFrame([tmp_country],columns=['date']+countries)
            )

    tmp_sheet.to_excel(writer, name)

writer.save()

In [4]:
tmp_sheet

,date,portugal,spain,italy,france,germany,uk,us,iran,china,south-korea
0,2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,NaN
1,2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN
3,2020-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,2020-01-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
68,2020-03-31,NaN,NaN,1109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,2020-04-01,NaN,NaN,1118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2020-04-02,NaN,NaN,1431.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,2020-04-03,NaN,NaN,1480.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
